In [1]:
needle_model_name = "NeedleModel"
sequence_browser_name = "NeedleCapture_01_SequenceBrowser"
image_name = "Image_Image"
image_copy_name = "Image_Image_3DEmpty"
segmentation_name = "Segmentation"

output_folder = "C:\Data\Temp"
filename_prefix = "NeedleCapture_01"

In [2]:
import datetime
import numpy as np
import os

In [5]:
browser_node = slicer.util.getFirstNodeByName(sequence_browser_name, className='vtkMRMLSequenceBrowserNode')
model_node = slicer.util.getFirstNodeByName(needle_model_name, className="vtkMRMLModelNode")
image_node = slicer.util.getFirstNodeByName(image_name, className="vtkMRMLScalarVolumeNode")
image_copy_node = slicer.util.getFirstNodeByName(image_copy_name, className="vtkMRMLScalarVolumeNode")
segmentation_node = slicer.util.getFirstNodeByName(segmentation_name, className="vtkMRMLSegmentationNode")


if browser_node is None:
    logging.error("Could not find input browser node: {}".format(sequence_browser_name))
    raise

if model_node is None:
    logging.error("Could not find input model node: {}".format(needle_model_name))
    raise

if image_node is None:
    logging.error("Could not find input image node: {}".format(image_name))
    raise
    
if image_copy_node is None:
    logging.error("Could not find input image node: {}".format(image_copy_name))
    raise

if segmentation_node is None:
    logging.error("Could not find input segmentation node: {}".format(segmentation_name))
    raise

In [6]:
labelmap_name = "temp_labelmap"
labelmap_node = slicer.util.getFirstNodeByName(labelmap_name, className="vtkMRMLLabelMapVolumeNode")
if labelmap_node is None:
    labelmap_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    labelmap_node.SetName(labelmap_name)

In [36]:
#get current image extent and spacing
image_extent = image_copy_node.GetImageData().GetExtent()
image_spacing = image_copy_node.GetSpacing()

#change extent of image (make thicker in z direction) and allocate necessary space
image_copy_node.GetImageData().SetExtent(image_extent[0], image_extent[1], image_extent[2], image_extent[3], -2, 2)
image_copy_node.GetImageData().AllocateScalars(image_copy_node.GetImageData().GetScalarType(), image_copy_node.GetImageData().GetNumberOfScalarComponents())

#shift so image extent starts at 0
image_copy_node.ShiftImageDataExtentToZeroStart()
print("Original Extent: {}\nModified Extent: {}".format(image_extent, image_copy_node.GetImageData().GetExtent()))

#increase image spacing
image_copy_node.SetSpacing(image_spacing[0], image_spacing[1], 2.0)
print("Original Spacing: {}\nModified Spacing: {}".format(image_spacing, image_copy_node.GetSpacing()))

Original Extent: (0, 511, 0, 511, 0, 4)
Modified Extent: (0, 511, 0, 511, 0, 4)
Original Spacing: (1.0, 1.0, 3.0)
Modified Spacing: (1.0, 1.0, 2.0)


In [15]:
import vtkSegmentationCorePython as vtkSegmentationCore

def removeSegments(selectedSegmentation):
    num_segments = selectedSegmentation.GetSegmentation().GetNumberOfSegments()
    for i in range(num_segments):
        segmentId = selectedSegmentation.GetSegmentation().GetNthSegmentID(i)
        selectedSegmentation.RemoveSegment(segmentId)

In [45]:
browser_node.SelectFirstItem()

segmentation_logic = slicer.modules.segmentations.logic()

notAllEmpty=False

image_cast = vtk.vtkImageCast()
image_cast.SetOutputScalarTypeToUnsignedChar()
image_cast.Update()

removeSegments(segmentation_node)
n = browser_node.GetNumberOfItems()

for i in range(n):
    
    #Get segmented image in US position
    segmentation_logic.ImportModelToSegmentationNode(model_node, segmentation_node)
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(segmentation_node, labelmap_node, image_copy_node)
    segmented_image = labelmap_node.GetImageData()
    
    image_cast.SetInputData(segmented_image)
    image_cast.Update()
    
    
    #Set ultrasound image filename
    img_file_name = filename_prefix + "_%04d_ultrasound" % i
    ultrasound_fullname = os.path.join(output_folder, img_file_name)
    
    #Set segmentation image filenames
    seg_file_name = filename_prefix + "_%04d_segmentation" % i
    segmentation_fullname = os.path.join(output_folder, seg_file_name)
    """
    #Output ultrasound image to file
    png_writer.SetInputData(image_node.GetImageData())
    png_writer.SetFileName(ultrasound_fullname)
    png_writer.Update()
    png_writer.Write()
    """
    
    ultrasound_array = slicer.util.arrayFromVolume(image_node)
    np.save(ultrasound_fullname, ultrasound_array)
    
    
    segmentation_array = slicer.util.arrayFromVolume(labelmap_node)
    np.save(segmentation_fullname, segmentation_array)
    if np.amax(segmentation_array) != 0:
        notAllEmpty = True
        #print("CHECK! " + segmentation_fullname + " " + str(np.amax(segmentation_array)))
    
    #Output segmentations
    removeSegments(segmentation_node)
    
    slicer.app.processEvents()
    browser_node.SelectNextItem()
    

CHECK! C:\Data\Temp\NeedleCapture_01_0111_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0112_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0113_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0193_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0197_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0198_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0222_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0223_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0224_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0225_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0226_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0227_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0228_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0229_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0230_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0231_segmentation 1
CHECK! C:\Data\Temp\NeedleCapture_01_0270_segmentation 1
CHECK! C:\Data\Temp\NeedleCaptu

In [39]:
#debug

recent_seg = np.load(segmentation_fullname + ".npy")
print(segmentation_fullname)
print(np.amax(recent_seg))
print(np.min(recent_seg))
print(recent_seg.shape)

recent_us = np.load(ultrasound_fullname + ".npy")
print(ultrasound_fullname)
print(np.amax(recent_us))
print(np.min(recent_us))
print(recent_us.shape)


C:\Data\Temp\NeedleCapture_01_0241_segmentation
0
0
(5, 512, 512)
C:\Data\Temp\NeedleCapture_01_0241_ultrasound
188
0
(1, 512, 512)
